In [72]:
#%matplotlib agg
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from pathlib import Path


In [74]:
export_dir_2 = "../data_comp-camp/Scatterplots/men_women"
export_dir_1 = "../data_comp-camp/Scatterplots/ordinary"

In [76]:
fig_width = 6.3  
fig_height = 3.54  

file_path = "D:/AI/Computerized-campimetry-test-analysis/summary_per_color_group.xlsx"
data = pd.read_excel(file_path)
necessary = data.rename(columns=lambda x: x.replace('T258', 'CM')) #замена названий столбцов

#columns = ["CM_dH+_yellow_green_50%", "CM_dH+_blue_50%","CM_dH+_purple_50%","CM_dH+_red_50%"]


## Построение графиков scatterplot со смещением точек для всех параметров с зависимостью от возраста

In [82]:
def build_age_param_scatterplots(test_data, columns, particular_column, save_fig=False, export_dir = None):
            
    #color_dict = sns.color_palette("bright", n_colors=4) 
    if export_dir is not None:
        export_dir = Path(export_dir)
        export_dir.mkdir(parents=True, exist_ok=True) #Создаёт папку по данному пути, первое создаёт все родительские папки,
                                                      #второе не вызовет ошибку если они уже есть
        
    for i,col in enumerate(columns):
        plt.figure(figsize=(fig_width, fig_height), dpi=300)
        
        upper_border = np.percentile(test_data[col], 99)
        low_border = np.percentile(test_data[col], 1)
        data = test_data[(test_data[col] <= upper_border) & (test_data[col] >= low_border)]
        
        sns.scatterplot(data=data, x=particular_column, y=col, color="blue", s=20) #s=50 размер точек до изменения

        lowess_res = sm.nonparametric.lowess(exog=data[particular_column], endog=data[col], frac=0.3)#продвинутое сглаживание точек, чем больше frac тем плавнее линия
                                                            #(доля точек для сглаживания), возвращает двумерный массив
        plt.plot(lowess_res[:, 0], lowess_res[:, 1], linewidth=2, color='black')
        
        col = col.replace('_', ' ') #изменение названий параметра для графика
        if (col == "CM dH+ yellow green 50%"):
            col = "CM dH+ yellow-green 50%"

        plt.xlabel('Возраст', fontsize=14, fontweight='light')
        plt.ylabel(col, fontsize=14, fontweight='light')
        plt.xticks(fontsize=14)  #размер шрифта цифр
        plt.yticks(fontsize=14)
       
    
        if save_fig:
            
            file_name = f'plot_{i}_{col}_vs_{particular_column}.png'
            plt.savefig(export_dir / file_name,format='png', bbox_inches='tight', dpi=300)   # объект path и имя файла объединяются в объект path


In [84]:
columns = necessary.columns[1:-4] #можно обращаться как к списку, save_fig=True, export_dir_1
build_age_param_scatterplots(necessary, columns, 'age',  save_fig=True, export_dir = export_dir_1)

In [58]:
def build_age_param_gender_scatterplots(test_data, columns, particular_column, save_fig=False, export_dir = None):
    
    if export_dir is not None:
        export_dir = Path(export_dir)
        export_dir.mkdir(parents=True, exist_ok=True) #Создаёт папку по данному пути, первое создаёт все родительские папки,
                                                      #второе не вызовет ошибку если они уже есть    
        
    for i,col in enumerate(columns):
        plt.figure(figsize=(fig_width, fig_height), dpi=300)
        
        upper_border = np.percentile(test_data[col], 99)
        low_border = np.percentile(test_data[col], 1)
        data = test_data[(test_data[col] <= upper_border) & (test_data[col] >= low_border)]
        
        mask_men = data["sex"] == 0
        mask_women = data["sex"] == 1
        
        sns.scatterplot(
            data=data[mask_men],
            x=data[particular_column][mask_men] + np.random.uniform(-0.5, 0.5, size=len(data[mask_men])),
            y=col,
            color='blue',
            s=20,
            label="Мужчины") #s=50 размер точек до изменения
        
        sns.scatterplot(
            data=data[mask_women],
            x=data[particular_column][mask_women] + np.random.uniform(-0.5, 0.5, size=len(data[mask_women])),
            y=col,
            color='red',
            s=20,
            label="Женщины") #s=50 размер точек до изменения
        
        lowess_res = sm.nonparametric.lowess(exog=data[particular_column], endog=data[col], frac=0.3)#продвинутое сглаживание точек, чем больше frac тем плавнее линия
                                                            #(доля точек для сглаживания), возвращает двумерный массив        
        plt.plot(lowess_res[:, 0], lowess_res[:, 1], linewidth=2, color='black')
        
        col = col.replace('_', ' ') #изменение названий параметра для графика
        if (col == "CM dH+ yellow green 50%"):
            col = "CM dH+ yellow-green 50%"

        plt.xlabel('Возраст', fontsize=14, fontweight='light')
        plt.ylabel(col, fontsize=14, fontweight='light')
        plt.xticks(fontsize=14)  #размер шрифта цифр
        plt.yticks(fontsize=14)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))# сдвиг легенды за график
        
        
        if save_fig:
            file_name = f'plot_{i}_{col}_vs_{particular_column}.png'
            plt.savefig(export_dir / file_name,format='png',  bbox_inches='tight', dpi=300)   # объект path и имя файла объединяются в объект path

In [60]:
#
build_age_param_gender_scatterplots(necessary, columns, 'age', save_fig=True, export_dir = export_dir_2)